In [1]:

# import pandas as pd
# import polars as pl

from dotenv import load_dotenv 
load_dotenv()

from langchain_core.output_parsers import PydanticOutputParser
from langchain.output_parsers import OutputFixingParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

from financialtools.wrappers import read_financial_results
from financialtools.pydantic_models import StockRegimeAssessment
from financialtools.prompts import system_prompt_StockRegimeAssessment
from financialtools.utils import get_sector_for_ticker, get_market_metrics, dataframe_to_json

In [2]:
ticker = 'ISP.MI'
# year = 2024

metrics, eval_metrics, composite_scores, red_flags = read_financial_results(
    ticker=ticker,
    # time=year,
    input_dir='financial_data', 
    sheet_name='sheet1')

In [3]:
sector = get_sector_for_ticker(ticker)

market_metrics = get_market_metrics(
    file_path='financial_data/metrics_by_sectors.xlsx',
    sector=sector)

eval_market_metrics = get_market_metrics(
    file_path='financial_data/eval_metrics_by_sectors.xlsx',
    sector=sector)

In [4]:
metrics, eval_metrics, composite_scores, red_flags, market_metrics, eval_market_metrics = [
    dataframe_to_json(df)
    for df in [metrics, eval_metrics, composite_scores, red_flags, market_metrics, eval_market_metrics]
]

In [5]:

# Instantiate the LLM (OpenAI GPT-4 or your preferred model)
llm = ChatOpenAI(model="gpt-4.1-nano", temperature=0)

# Instantiate the parser with the Pydantic model
parser = PydanticOutputParser(pydantic_object=StockRegimeAssessment)
# Wrap your parser with OutputFixingParser
parser = OutputFixingParser.from_llm(parser=parser, llm=llm)

# Get the format instructions string from the parser
format_instructions = parser.get_format_instructions()

# Create a ChatPromptTemplate with system message and user input

system_prompt_filled = system_prompt_StockRegimeAssessment.format(format_instructions=format_instructions)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt_filled),
    ("human", "Metrics:\n{metrics}\nScores:\n{composite_scores}\nEvaluation Metrics:\n{eval_metrics}\nRedFlags:\n{red_flags}\nMarket metrics:{market_metrics}\nMarket evaluation metrics:{eval_market_metrics}"),
])

# Create a runnable chain: prompt followed by LLM invocation
chain = prompt | llm | parser


# Then invoke with a dict containing 'financial_data'
response = chain.invoke({
    "metrics": metrics,  
    "eval_metrics": eval_metrics,
    "composite_scores": composite_scores,
    "red_flags": red_flags,   
    "market_metrics": market_metrics, 
    "eval_market_metrics": eval_market_metrics
})


In [6]:
import rich

rich.print(response)

StockRegimeAssessment(
    ticker='XYZ',
    regime='bear',
    regime_rationale="The company's fundamentals have weakened significantly, with declining composite scores, high
and increasing debt-to-equity ratios exceeding 2, and negative cash flows in recent years. These indicators point 
to financial distress and increased risk, justifying a bear regime.",
    metrics_movement='Composite score declined from 3.0 in 2020 to 2.14 in 2024, debt-to-equity ratio increased 
above 2, free cash flow remained negative in 2023 and 2024, profit margins and return ratios stayed modest, cash 
flow metrics deteriorated.',
    non_aligned_findings=None,
    evaluation='undervaluated',
    evaluation_rationale='Despite being undervalued relative to sector averages, the deteriorating financial 
health, high leverage, and negative cash flows indicate increased risk, justifying an undervalued assessment with 
caution.',
    market_comparison="The stock's valuation metrics are below sector averages, but the declining fundamentals and 
red flags suggest caution despite the undervaluation."
)

In [1]:
import rich
from financialtools.chains import get_stock_evaluation_report
ticker = 'AVIO.MI'

rich.print(get_stock_evaluation_report(ticker)
)
    

StockRegimeAssessment(
    ticker='XYZ',
    regime='bear',
    regime_rationale="The company's financial metrics show declining profitability, high valuation ratios, and red 
flags such as negative net profit margin, ROA, and ROE in 2022. Despite some improvements in cash flow and debt 
levels in 2024, overall fundamentals remain weak, indicating a bearish regime.",
    metrics_movement='GrossMargin declined, DebtToEquity increased, NetProfitMargin turned negative in 2022, ROA 
and ROE dropped sharply, FCFYield remained low, and P/E ratios stayed high.',
    non_aligned_findings=None,
    evaluation='overvaluated',
    evaluation_rationale='High valuation ratios combined with weak profitability metrics and red flags suggest the 
stock is overvalued and not supported by strong fundamentals.',
    market_comparison="The company's profitability and cash flow metrics lag behind sector averages, reinforcing 
concerns about overvaluation and weak financial health."
)